In [2]:
from read_db_json import *

In [3]:
from sklearn.metrics import r2_score, mean_squared_error

In [4]:
date = '2017-09-29'

In [5]:
ratings = large_joint_table(date=date)

with test (853, 12)
without test users (853, 12)


In [6]:
ratings.shape

(1346, 31)

In [7]:
y_all = weighted_seg_score(ratings)

In [8]:
y_all.shape

(407, 1)

In [10]:
y_all.to_csv('ys/%s/y_csl_all.csv' % date)

# find sub type worth experimenting

In [7]:
def get_stat(filtered_ratings, col, label):
    if filtered_ratings.shape[0]==0:
        return {'question': col, 'answer':label, '#data points': 0}
    else:
        cmp_all = y_all.join(filtered_ratings, lsuffix='_all', how='inner')    
        r2 = r2_score(cmp_all.csl_all, cmp_all.csl)
        rmse = pd.np.sqrt(mean_squared_error(cmp_all.csl_all, cmp_all.csl))
        same_class_pcnt = (cmp_all.csl_all.round()==cmp_all.csl.round()).mean()
        return {
            'question': col, 'answer': label, '#data points':filtered_ratings.shape[0],
            'r2' : r2, 'rmse': rmse, 'same_class_percentage': same_class_pcnt,
        }

In [8]:
y_shape_with_filtered = []
for col, labels in SURVEY_DICT.items():
    question_col = ratings[col].fillna(0).astype(int)
    for i, label in enumerate(labels):  
        filtered_ratings = weighted_seg_score(ratings[question_col==i])
        stat = get_stat(filtered_ratings, col, label)
        y_shape_with_filtered.append(stat)

col = 'experienceLevel'
question_col = ratings[col].fillna('No Info')
for label in ['No Info', 'fearless', 'confident', 'interested', 'reluctant']:
    filtered_ratings = weighted_seg_score(ratings[question_col==label])
    stat = get_stat(filtered_ratings, col, label)
    y_shape_with_filtered.append(stat)
    

In [9]:
df_y_shape=pd.DataFrame(y_shape_with_filtered)

In [28]:
df_y_shape[['question', 'answer', '#data points', 'r2', 'rmse', 'same_class_percentage']].to_csv('filtered_y_shape.csv')

# get sub ys

In [6]:
sub_ys = {}

for col, labels in SURVEY_DICT.items():
    # consider these 4 types+exp lvl only
    if col not in ['bk_purpose', 'household_income', 'age', 'bk_type']:
        continue
    question_col = ratings[col].fillna(0).astype(int)
    for i, label in enumerate(labels):  
        # exclude no Info
        if i==0:
            continue
        filtered_ratings = weighted_seg_score(ratings[question_col==i])
        if filtered_ratings.shape[0]>30:
            sub_ys[(col, label)] = filtered_ratings

col = 'experienceLevel'
question_col = ratings[col].fillna('No Info')
for label in ['No Info', 'fearless', 'confident', 'interested', 'reluctant']:
    filtered_ratings = weighted_seg_score(ratings[question_col==label])
    if filtered_ratings.shape[0]>30:
        sub_ys[(col, label)] = filtered_ratings

In [7]:
sub_ys.keys()

dict_keys([('bk_purpose', 'Mainly utility biking – traveling from one location to another'), ('bk_purpose', 'Mainly recreational biking – biking for recreation, leisure, and health'), ('bk_purpose', '50% utility, 50% recreational'), ('age', '25-34'), ('age', '35-44'), ('household_income', 'Less than $24,999'), ('household_income', '$25,000 to $39,999'), ('household_income', '$55,000 to $69,999'), ('household_income', '$70,000 to $84,999'), ('household_income', '$85,000 to $99,999'), ('household_income', '$100,000 to $149,999'), ('household_income', '$150,000 or more'), ('bk_type', 'Road Bike'), ('bk_type', 'Hybrid Bike'), ('experienceLevel', 'fearless'), ('experienceLevel', 'confident')])

In [16]:
for key, df in sub_ys.items():
    fn = 'ys/y_csl_%s#%s.csv' % (key[0], key[1].split('–')[0])
    df.to_csv(fn, encoding='utf8')